In [29]:
from typing import Literal, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.graph import END

from dotenv import load_dotenv
import os

from langchain.agents import Tool
from langchain.tools import StructuredTool
from notion.planner import Planner, ConfirmablePlanner
from notion.task import Task
from pydantic import BaseModel, Field

from datetime import datetime

load_dotenv()

# 메모리 저장소 설정
memory = MemorySaver()

# planner = Planner()
planner= ConfirmablePlanner()

# 메시지 상태와 요약 정보를 포함하는 상태 클래스
class State(MessagesState):
    messages: Annotated[list, add_messages]
    summary: str


# 대화 및 요약을 위한 모델 초기화
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


# planner 도구

class ShowTasksSchema(BaseModel):
    pass

def show_tasks():
    return planner.show_tasks()

class ShowTasksByGroupSchema(BaseModel):
    group: str = Field(..., description="The group to filter tasks by, e.g., '일상', '이벤트'")

def show_tasks_by_group(group: str):
    return planner.show_tasks_by_group(group)

class ShowTasksAfterNowSchema(BaseModel):
    pass

def show_tasks_after_now():
    return planner.show_tasks_after_now()

class CreateTaskSchema(BaseModel):
    name: str = Field("No name", description="Task name")
    date: str = Field("2000-01-01 00:00", description="The date of the task, either format:\n- Single datetime: 'YYYY-MM-DD HH:MM'\n- Start and end datetime: 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'")
    group: str = Field("no group", description="Task group, e.g., '일상', '이벤트'")

def create_task(name, date, group):
    try:
        date_count = date.count(":")
        if date_count == 1:
            date = {"start": date, "end": None}
        elif date_count == 2:
            part = date.split(" ")
            start = f"{part[0]} {part[1]}"
            end = f"{part[2]} {part[3]}"
            date = {"start": start, "end": end}
        else:
            raise ValueError("Invalid date format. Use 'YYYY-MM-DD HH:MM' or 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'.")
    except ValueError as e:
        raise ValueError(f"Error parsing date: {e}")

    task = Task(task_id=None, name=name, date=date, group=group)
    return planner.add_task(task)

class DeleteTaskSchema(BaseModel):
    task_id: str = Field(..., description="The ID of the task to delete")

def delete_task(task_id):
    return planner.delete_task(task_id)

class EditTaskSchema(BaseModel):
    task_id: str = Field(..., description="The ID of the task to edit")
    name: str = Field("No name", description="New task name")
    date: str = Field("2000-01-01 00:00", description="New date of the task, either format:\n- Single datetime: 'YYYY-MM-DD HH:MM'\n- Start and end datetime: 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'")
    group: str = Field("no group", description="New task group, e.g., '일상', '이벤트'")

def edit_task(task_id, name, date, group):
    new_task = Task(task_id=task_id, name=name, date=date, group=group)
    return planner.edit_task(task_id, new_task)


# 기타 도구
class GetNowDatetimeSchema(BaseModel):
    pass

def get_now_datetime():
    return datetime.now().astimezone().strftime('%Y-%m-%d %H:%M:%S %Z')


tools = [
    StructuredTool.from_function(
        func=show_tasks,
        name="ShowTasks",
        description="Show all the tasks.",
        args_schema=ShowTasksSchema
    ),
    StructuredTool.from_function(
        func=show_tasks_by_group,
        name="ShowTasksByGroup",
        description="Show tasks filtered by group.",
        args_schema=ShowTasksByGroupSchema
    ),
    StructuredTool.from_function(
        func=show_tasks_after_now,
        name="ShowTasksAfterNow",
        description="Show tasks that are scheduled after the current time.",
        args_schema=ShowTasksAfterNowSchema
    ),
    StructuredTool.from_function(
        func=create_task,
        name="CreateTask",
        description="Create/Add a new task in Notion.",
        args_schema=CreateTaskSchema
    ),
    StructuredTool.from_function(
        func=delete_task,
        name="DeleteTask",
        description="Delete a task from Notion. Input: Notion page ID.",
        args_schema=DeleteTaskSchema
    ),
    StructuredTool.from_function(
        func=edit_task,
        name="EditTask",
        description="Update a task's title in Notion. Input: page ID and new title.",
        args_schema=EditTaskSchema
    ),

    StructuredTool.from_function(
        func=get_now_datetime,
        name="GetNowDatetime",
        description="Get the current date and time in the format 'YYYY-MM-DD HH:MM:SS TZ'.",
        args_schema=GetNowDatetimeSchema
    ),
]

def print_stream(stream):
    for chunk in stream:
        message = chunk["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    model=model,
    tools=tools,
    # checkpointer=memory
)

In [7]:
"""
config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [
    ("user", "how much tasks did you saw?")
]}
response = agent_executor.invoke(
    inputs
)
print(response["messages"][-1].content)

config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [
    ("user", "how much tasks did you saw?")
]}
print_stream(agent_executor.stream(inputs, config=config, stream_mode="values"))
"""

'\nconfig = {"configurable": {"thread_id": "1"}}\ninputs = {"messages": [\n    ("user", "how much tasks did you saw?")\n]}\nresponse = agent_executor.invoke(\n    inputs\n)\nprint(response["messages"][-1].content)\n\nconfig = {"configurable": {"thread_id": "1"}}\ninputs = {"messages": [\n    ("user", "how much tasks did you saw?")\n]}\nprint_stream(agent_executor.stream(inputs, config=config, stream_mode="values"))\n'

In [30]:
def ask_llm(state: State):
    # 이전 요약 정보 확인
    summary = state.get("summary", "")

    # 이전 요약 정보가 있다면 시스템 메시지로 추가
    if summary:
        # 시스템 메시지 생성
        system_message = f"Summary of conversation earlier: {summary}"
        # 시스템 메시지와 이전 메시지 결합
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        # 이전 메시지만 사용
        messages = state["messages"]

    # 모델 호출
    # response = model.invoke(messages)
    messages = {'messages': messages}
    response = agent_executor.invoke(messages)['messages'][-1]

    # 응답 반환
    return {"messages": [response]}


# 대화 종료 또는 요약 결정 로직
def should_continue(state: State) -> Literal["summarize_conversation", END]:
    # 메시지 목록 확인
    messages = state["messages"]

    # 메시지 수가 6개 초과라면 요약 노드로 이동
    if len(messages) > 6:
        return "summarize_conversation"
    return END

# 대화 내용 요약 및 메시지 정리 로직
def summarize_conversation(state: State):
    # 이전 요약 정보 확인
    summary = state.get("summary", "")

    # 이전 요약 정보가 있다면 요약 메시지 생성
    if summary:
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above in Korean:"
        )
    else:
        # 요약 메시지 생성
        summary_message = "Create a summary of the conversation above in Korean:"

    # 요약 메시지와 이전 메시지 결합
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    # 모델 호출
    response = model.invoke(messages)
    # 오래된 메시지 삭제
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    # 요약 정보 반환
    return {"summary": response.content, "messages": delete_messages}

# 업데이트 정보 출력 함수
def print_update(update):
    # 업데이트 딕셔너리 순회
    for k, v in update.items():
        # 메시지 목록 출력
        for m in v["messages"]:
            m.pretty_print()
        # 요약 정보 존재 시 출력
        if "summary" in v:
            print(v["summary"])


# 워크플로우 그래프 초기화
workflow = StateGraph(State)

# 대화 및 요약 노드 추가
workflow.add_node("conversation", ask_llm)
workflow.add_node(summarize_conversation)

# 시작점을 대화 노드로 설정
workflow.add_edge(START, "conversation")

# 조건부 엣지 추가
workflow.add_conditional_edges(
    "conversation",
    should_continue,
)

# 요약 노드에서 종료 노드로의 엣지 추가
workflow.add_edge("summarize_conversation", END)

# 워크플로우 컴파일 및 메모리 체크포인터 설정
app = workflow.compile(checkpointer=memory)

print(app.get_graph().draw_ascii())

                      +-----------+           
                      | __start__ |           
                      +-----------+           
                             *                
                             *                
                             *                
                     +--------------+         
                     | conversation |         
                     +--------------+         
                    ...             ...       
                  ..                   ...    
                ..                        ..  
+------------------------+                  ..
| summarize_conversation |                ..  
+------------------------+             ...    
                    ***             ...       
                       **         ..          
                         **     ..            
                        +---------+           
                        | __end__ |           
                        +---------+           


In [34]:
# 메시지 핸들링을 위한 HumanMessage 클래스 임포트
from langchain_core.messages import HumanMessage

# 스레드 ID가 포함된 설정 객체 초기화
config = {"configurable": {"thread_id": "1"}}

# 첫 번째 사용자 메시지 생성 및 출력
input_message = HumanMessage(content="test 일정 없애")
input_message.pretty_print()

# 스트림 모드에서 첫 번째 메시지 처리 및 업데이트 출력
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)


================================ Human Message =================================

test 일정 없애
Action[Delete Task] has been requested.
Action info: 
Task ID: edit_test
Action[Delete Task] has been requested.
Action info: 
Task ID: 1cc84e0b-d49f-8173-a2e0-c9c83841a781
================================== Ai Message ==================================

"test" 일정인 **edit_test**가 성공적으로 삭제되었습니다. 다른 도움이 필요하시면 말씀해 주세요!
================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


대화 요약:

사용자는 현재 시간을 물었고, AI는 2025년 6월 21일 19시 29분 51초라고 답변했다. 이후 사용자는 이미 지난 일정을 보여달라고 요청했으며